In [2]:

import pandas as pd



# read matrix
with open(
    "/data/LSY/z_preparing_and_parts/commontrait_Erythematosus/GSE59250_series_matrix.txt", "r"
) as file:
    lines = file.readlines()

# find metadata
first_empty_line_index = lines.index("\n")
table_begin_index = next(
(
        i
        for i, line in enumerate(lines)
        if line.startswith("!series_matrix_table_begin")
    ),
    None,
)
end_index = table_begin_index if table_begin_index is not None else len(lines)
content_before_table = lines[first_empty_line_index:end_index]


# read metadata
metadata_lines = [
    line.strip()[1:].split("\t")
    for line in content_before_table
    if line.startswith("!")
]
metadata_df = pd.DataFrame(metadata_lines).T


metadata_df.columns = metadata_df.iloc[0]
metadata_df = metadata_df.drop(0, axis=0)

metadata_df.dropna(inplace=True)


In [41]:
# mask = metadata_df.iloc[:, 7] == '"Peripheral blood"'

# metadata_df = metadata_df[mask]



In [6]:
print(metadata_df.iloc[:,10].unique())

['"disease state: control"'
 '"disease state: Systemic Lupus Erythematosus"']


In [7]:

# find status
metadata_df["status"] = metadata_df.iloc[:, 10].apply(
    lambda x: 1 if "Erythematosus" in x else 0 if "control" in x else None
)
metadata_df = metadata_df.set_index("Sample_geo_accession")

In [8]:

metadata_df = metadata_df["status"].apply(pd.to_numeric, errors="coerce")

In [9]:
print(metadata_df)

Sample_geo_accession
"GSM1431412"    0
"GSM1431413"    1
"GSM1431414"    1
"GSM1431415"    1
"GSM1431416"    1
               ..
"GSM1431865"    1
"GSM1431866"    1
"GSM1431867"    0
"GSM1431868"    0
"GSM1431869"    1
Name: status, Length: 434, dtype: int64


In [10]:


# read data
data_lines = [line.strip().split() for line in lines if not line.startswith("!")]
data_df = pd.DataFrame(data_lines).T

data_df.columns = data_df.iloc[0]
data_df = data_df.drop(0, axis=0)

data_df = data_df.set_index('"ID_REF"')
data_df = data_df.iloc[:, 1:]
data_df.columns = [col.strip('"') for col in data_df.columns]

data_df = data_df.apply(pd.to_numeric, errors="coerce")





In [11]:
print(data_df)

              cg00000029  cg00000108  cg00000109  cg00000165  cg00000236  \
"ID_REF"                                                                   
"GSM1431412"    0.497578    0.963640    0.914327    0.249481    0.746413   
"GSM1431413"    0.571429    0.955141    0.891096    0.210200    0.725640   
"GSM1431414"    0.503639    0.965829    0.909776    0.229172    0.716118   
"GSM1431415"    0.451789    0.953644    0.914833    0.254187    0.765189   
"GSM1431416"    0.489838    0.956569    0.940952    0.231415    0.730905   
...                  ...         ...         ...         ...         ...   
"GSM1431865"    0.721787    0.944998    0.893127    0.286073    0.880319   
"GSM1431866"    0.761533    0.959776    0.881931    0.284751    0.905025   
"GSM1431867"    0.728761    0.935058    0.896056    0.353870    0.863559   
"GSM1431868"    0.717342    0.921975    0.793701    0.279716    0.838519   
"GSM1431869"    0.832698    0.955981    0.899794    0.386179    0.911444   

           

In [9]:

def m_to_beta(M):
    return 1 / (1 + 2 ** (-M))


In [10]:
# data_df = data_df.apply(m_to_beta)

In [12]:

# merge metadata and data
data_df["status"] = metadata_df
# drop other disease sample
data_df.dropna(subset=["status"], inplace=True)

In [13]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data_df, test_size=0.2, random_state=42)


# save data
train_data.to_csv(
    "/data/LSY/z_preparing_and_parts/commontrait_Erythematosus/data_train.txt",
    index=True,
    header=True,
    sep="\t",
)


# save data
test_data.to_csv(
    "/data/LSY/z_preparing_and_parts/commontrait_Erythematosus/data_test.txt",
    index=True,
    header=True,
    sep="\t",
)
